**Introduction**

This Google Colab notebook provides a user-friendly interface for remotely controlling and monitoring PioReactors, designed to facilitate yeast growth studies in various ethanol concentrations. PioReactors enable controlled environmental conditions for microbial growth experiments, particularly useful for studying yeast's ethanol tolerance and the effects of different alcohol concentrations on yeast metabolism. By using Google Colab, you can execute commands, automate tasks, and view data from the PioReactor directly in this notebook without requiring physical interaction with the device. Although physical interaction will be need to change vials, add ingredients, etc.

This is based of [yeast- in-ethanol](https://docs.pioreactor.com/experiments/yeast-in-ethanol).

Install dependencies to allow for correct usage

In [ ]:
pip install gradio_client

Apply imports and define client

In [ ]:
from gradio_client import Client

client = Client("AccelerationConsortium/PioReactor_gradio")

API call to check the the current status of the PioReactor

In [ ]:
result = client.predict(
		exp="Ed",
		api_name="/get_status_default"
)
print(result)

experiment = result[0] # Stores the current experiment from the results of the API call

Create a new experiment for our example workflow and switch to it

In [ ]:
new_experiment = "EXPERIMENT_NAME_HERE" # Change this to what you would like

result = client.predict(
		new_exp=new_experiment,
        exp=experiment,
		api_name="/new_experiment_default"
)

result = client.predict(
		ch_exp=new_experiment,
		exp=experiment,
		api_name="/change_experiment_default"
)

experiment = new_experiment

The pials should now be prepared. Follow the detailed procecures here [yeast- in-ethanol](https://docs.pioreactor.com/experiments/yeast-in-ethanol).

After you after done, execute the next block to start stirring and OD reading, you can optionally turn heating on by changing the boolean.

In [ ]:
heating = False
temp=30

rpm_speed = 500

result = client.predict(
		rpm=rpm_speed,
		experiment=experiment,
		state="start",
		api_name="/stirring_default"
)

result = client.predict(
		experiment=experiment,
		state="start",
		api_name="/od_default"
)

if heating:
    result = client.predict(
		temperature=30,
		experiment=experiment,
		state="start",
		option="Heat To Temp",
		api_name="/temp_automation_default"
)

After execution, you must wait for a desierd amount of time, aprox 18 hours. After which you can execute the code block below to stock the data logging.
You can change the variables of the experiment and start again.

In [ ]:
result = client.predict(
		rpm=0,
		experiment=experiment,
		state="stop",
		api_name="/stirring_default"
)

result = client.predict(
		experiment=experiment,
		state="stop",
		api_name="/od_default"
)

if heating:
    result = client.predict(
		temperature=30,
		experiment=experiment,
		state="stop",
		option="Heat To Temp",
		api_name="/temp_automation_default"
)

You can view the graphs by using the APIs below.

In [ ]:
import plotly.graph_objs as go

# Time frame you want to see the data for (1 hour, 24 hour, All Data)
amount = "All Data"

result = client.predict(
		time_scale=amount,
		exp=experiment
		api_name="/get_data_default"
)

# The results of the API call are go.Figure objects
# You can use the plotly library to display the data

result[0].show()